In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Sun Dec 26 19:02:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16280356804607292780
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11320098816
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6327530176811494233
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
 xla_global_id: 416903419]

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/test.csv')
train['data'][0]

'신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지 마시고\\n보편적으로 모든국민이 수긍할  수 있는 복지정책 펴 주시길 바랍니다.\\n저도 신혼부부이지만 당첨되는 사람 로또되는 이런주택정책 반대합니다.\\n국민세금을 일부 사람들에게 퍼주기식이 되면 안되죠..\\n그 세금으로 우리아이 안전하게 맡길 수 있는 보육시설을 전국에 설치해 주세요..\\n대기업들은 솔선수범해서 모든 사업장에 의무설치 할 수 있도록 하시구요..\\n집 보다 애 맡길데가 없어 경력단절 되는게 더 괴롭습니다.!\\n집은 개인의 능력을 키워 사는게 맞습니다.\\n그 능력을 키울수 있도록 육아 전담에 힘을 기울이는게 맞습니다.\\n우리아이 부모가 키우는거 맞지만 이제는 국가가\\n책임지는 시대로 가는게 맞다고 봅니다.\\n그렇잖아도 부동산 가격 자꾸 올라가는게 정부정책이 잘못 되었다고 봅니다.\\n부동산은 그냥 내버려 두세요!  좀!\\n건들수록 역효과네요..'

## 1. 전처리

In [ ]:
train = train[train['data'].notnull()]
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39992 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     39992 non-null  int64 
 1   category  39992 non-null  int64 
 2   data      39992 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.2+ MB


In [ ]:
train['data'] = train['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test['data'] = test['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
stopwords = ['의','가','이','은','들','는','좀','잘','걍',
             '과','도','를','으로','자','에','와','한','하다','을',
             '더','좀','자꾸','그냥','이제','우리']

In [ ]:
train['data'][0]

'신혼부부위한 주택정책 보다 보육시설 늘려주세요 국민세금으로 일부를 위한 정책펴지 마시고보편적으로 모든국민이 수긍할  수 있는 복지정책 펴 주시길 바랍니다저도 신혼부부이지만 당첨되는 사람 로또되는 이런주택정책 반대합니다국민세금을 일부 사람들에게 퍼주기식이 되면 안되죠그 세금으로 우리아이 안전하게 맡길 수 있는 보육시설을 전국에 설치해 주세요대기업들은 솔선수범해서 모든 사업장에 의무설치 할 수 있도록 하시구요집 보다 애 맡길데가 없어 경력단절 되는게 더 괴롭습니다집은 개인의 능력을 키워 사는게 맞습니다그 능력을 키울수 있도록 육아 전담에 힘을 기울이는게 맞습니다우리아이 부모가 키우는거 맞지만 이제는 국가가책임지는 시대로 가는게 맞다고 봅니다그렇잖아도 부동산 가격 자꾸 올라가는게 정부정책이 잘못 되었다고 봅니다부동산은 그냥 내버려 두세요  좀건들수록 역효과네요'

---

### keyword 뽑아내기
1. ***명사 - nouns***
2. 형태소(어간) - morphs
- 긍/부정이 아니라 카테고리만 분류하는 것이므로 명사 뽑아내기

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 86 kB 6.4 MB/s 
     |████████████████████████████████| 448 kB 33.4 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from konlpy.tag import (Kkma, Okt)
kkma = Kkma()
okt = Okt()

In [ ]:
print(okt.morphs(train['data'][0]))

['신혼부부', '위', '한', '주택', '정책', '보다', '보육', '시설', '늘려주세요', '..', '국민', '세금', '으로', '일부', '를', '위', '한', '정책', '펴지', '마시고', '\\', 'n', '보편', '적', '으로', '모든', '국민', '이', '수긍', '할', '수', '있는', '복지', '정책', '펴', '주시길', '바랍니다', '.\\', 'n', '저', '도', '신혼부부', '이지만', '당첨', '되는', '사람', '로또', '되는', '이런', '주택', '정책', '반대', '합니다', '.\\', 'n', '국민', '세금', '을', '일부', '사람', '들', '에게', '퍼', '주기', '식이', '되면', '안되죠', '..\\', 'n', '그', '세금', '으로', '우리', '아이', '안전하게', '맡길', '수', '있는', '보육', '시설', '을', '전국', '에', '설치', '해', '주세요', '..\\', 'n', '대기업', '들', '은', '솔선수범', '해서', '모든', '사업', '장', '에', '의무', '설치', '할', '수', '있도록', '하시구요', '..\\', 'n', '집', '보다', '애', '맡길데가', '없어', '경력', '단절', '되는게', '더', '괴롭습니다', '.!\\', 'n', '집은', '개인', '의', '능력', '을', '키워', '사는게', '맞습니다', '.\\', 'n', '그', '능력', '을', '키울수', '있도록', '육아', '전담', '에', '힘', '을', '기울이는게', '맞습니다', '.\\', 'n', '우리', '아이', '부모', '가', '키우는거', '맞지만', '이제', '는', '국가', '가', '\\', 'n', '책임지는', '시대', '로', '가는게', '맞다고', '봅니다', '.\\', 'n', '그렇잖아도', '부동산', '가격', '자

In [ ]:
print(kkma.nouns(train['data'][0]))

['신혼부부', '신혼부부위', '위', '주택', '주택정책', '정책', '보육', '보육시설', '시설', '주세', '국민', '국민세금', '세금', '일부', '보편적', '수긍', '수', '복지', '복지정책', '신혼부부이지만', '이지만', '당첨', '사람', '로또', '반대', '주기', '주기식이', '식이', '우리', '아이', '안전', '전국', '설치', '대기업', '솔선수범', '사업장', '의무', '의무설치', '집', '애', '데', '경력', '경력단절', '단절', '개인', '능력', '육아', '전담', '힘', '부모', '거', '이제', '국가', '시대', '부동산', '가격', '정부', '정부정책', '수록', '역', '역효과', '효과']


In [ ]:
print(okt.nouns(train['data'][0])) # 선택

['신혼부부', '위', '주택', '정책', '보육', '시설', '국민', '세금', '일부', '위', '정책', '보편', '국민', '수긍', '수', '복지', '정책', '저', '신혼부부', '당첨', '사람', '로또', '주택', '정책', '반대', '국민', '세금', '일부', '사람', '퍼', '주기', '식이', '그', '세금', '우리', '아이', '수', '보육', '시설', '전국', '설치', '대기업', '솔선수범', '모든', '사업', '의무', '설치', '수', '집', '애', '경력', '단절', '더', '개인', '능력', '그', '능력', '육아', '전담', '힘', '우리', '아이', '부모', '이제', '국가', '시대', '부동산', '가격', '자꾸', '정부', '정책', '잘못', '부동산', '그냥', '좀', '수록', '역효과']


In [ ]:
from collections import Counter
print(Counter(okt.nouns(train['data'][0])))

Counter({'정책': 5, '국민': 3, '세금': 3, '수': 3, '신혼부부': 2, '위': 2, '주택': 2, '보육': 2, '시설': 2, '일부': 2, '사람': 2, '우리': 2, '아이': 2, '설치': 2, '능력': 2, '부동산': 2, '보편': 1, '수긍': 1, '복지': 1, '도': 1, '당첨': 1, '로또': 1, '반대': 1, '퍼': 1, '주기': 1, '식이': 1, '전국': 1, '대기업': 1, '솔선수범': 1, '모든': 1, '사업': 1, '의무': 1, '집': 1, '애': 1, '경력': 1, '단절': 1, '더': 1, '개인': 1, '육아': 1, '전담': 1, '힘': 1, '부모': 1, '이제': 1, '국가': 1, '시대': 1, '가격': 1, '자꾸': 1, '정부': 1, '잘못': 1, '그냥': 1, '좀': 1, '수록': 1, '역효과': 1})


In [ ]:
# 저장한 X_test, X_train 불러옴

from tqdm import tqdm

X_train = []
for sentence,i in zip(train['data'],tqdm(range(len(train['data'])))):
    temp_X = []
    temp_X = okt.nouns(sentence) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    temp_X = [word for word in temp_X if len(word) > 1]
    X_train.append(temp_X)

 13%|█▎        | 5000/39992 [06:35<46:09, 12.64it/s]


KeyboardInterrupt: ignored

In [ ]:
X_test = []

for sentence,i in zip(test['data'],tqdm(range(len(test['data'])))):
    temp_X = []
    temp_X = okt.nouns(sentence) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    temp_X = [word for word in temp_X if len(word) > 1]
    X_test.append(temp_X)

In [ ]:
import csv
X_test = list()

f = open("/content/drive/MyDrive/data/X_test.csv",'r',encoding='cp949')
rea = csv.reader(f)
for row in rea:
    X_test.append(row)
f.close

<function TextIOWrapper.close>

In [ ]:
len(X_test)

5000

In [ ]:
import csv
X_train = list()

f = open("/content/drive/MyDrive/data/X_train.csv",'r',encoding='cp949')
rea = csv.reader(f)
for row in rea:
    X_train.append(row)
f.close

<function TextIOWrapper.close>

In [ ]:
len(X_train)

39992

In [ ]:
print(X_train[:2])

[['신혼부부', '주택', '정책', '보육', '시설', '국민', '세금', '일부', '정책', '보편', '국민', '수긍', '복지', '정책', '신혼부부', '당첨', '사람', '로또', '주택', '정책', '반대', '국민', '세금', '일부', '사람', '주기', '식이', '세금', '아이', '보육', '시설', '전국', '설치', '대기업', '솔선수범', '모든', '사업', '의무', '설치', '경력', '단절', '개인', '능력', '능력', '육아', '전담', '아이', '부모', '국가', '시대', '부동산', '가격', '정부', '정책', '잘못', '부동산', '수록', '역효과'], ['학교', '이름', '남자', '울산', '여자', '중학교', '재학', '학생', '최근', '양성평등', '글짓기', '생각', '울산', '울산', '중학교', '개입', '하나', '남중', '하나', '여중', '어째서', '학교', '여자', '남자', '우위', '학교', '울산', '중학교', '남중', '남자', '산남', '중학교', '평소', '학교', '이름', '학교', '여중', '남중', '동안', '현실', '지금', '조선시대', '국내', '학교', '널렸습니', '울산', '국내', '다른', '고등학교', '문제', '해결']]


---

Tokenizer
- num_words : 단어 빈도가 많은 순서로 num_words개의 단어를 보존

In [ ]:
/content/drive/MyDrive/data/


---

In [ ]:
import os

os.getcwd()

'/content'

In [ ]:
os.chdir('/content/drive/MyDrive/data/KoBERT-master')
os.getcwd()

'/content/drive/MyDrive/data/KoBERT-master'

In [ ]:
pip install -r requirements.txt

In [ ]:
pip install .

ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
device = torch.device("cuda:0")

In [ ]:
#bert 모델, vocab 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/drive/My Drive/data/KoBERT-master/.cache/kobert_v1.zip
using cached model. /content/drive/My Drive/data/KoBERT-master/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
train[:5]

,index,category,data
0,0,2,신혼부부위한 주택정책 보다 보육시설 늘려주세요 국민세금으로 일부를 위한 정책펴지 마...
1,1,0,학교이름에 남자도 붙여주세요 울산여자중학교에 재학중인 학생입니다 최근 양성평등 글짓...
2,2,1,빙상연맹 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요 최근 동계올림픽에...
3,3,1,티비 세세 관람가도 연령확인 의무화 하자 제기 에전에 티비를 보다가 잠시 딴일이 생...
4,4,1,무더운 여름철엔 남성들도 시원한 자율복장을 해야 무더운 여름철에는 남성들도 노넥타이...


In [ ]:
dataset_train = nlp.data.TSVDataset('/content/drive/MyDrive/data/train_train.tsv', 
                                    field_indices=[2,1], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/content/drive/MyDrive/data/train_validation.tsv', 
                                   field_indices=[2,1], num_discard_samples=1)


In [ ]:
dataset_train[0]

['신혼부부위한 주택정책 보다 보육시설 늘려주세요 국민세금으로 일부를 위한 정책펴지 마시고보편적으로 모든국민이 수긍할  수 있는 복지정책 펴 주시길 바랍니다저도 신혼부부이지만 당첨되는 사람 로또되는 이런주택정책 반대합니다국민세금을 일부 사람들에게 퍼주기식이 되면 안되죠그 세금으로 우리아이 안전하게 맡길 수 있는 보육시설을 전국에 설치해 주세요대기업들은 솔선수범해서 모든 사업장에 의무설치 할 수 있도록 하시구요집 보다 애 맡길데가 없어 경력단절 되는게 더 괴롭습니다집은 개인의 능력을 키워 사는게 맞습니다그 능력을 키울수 있도록 육아 전담에 힘을 기울이는게 맞습니다우리아이 부모가 키우는거 맞지만 이제는 국가가책임지는 시대로 가는게 맞다고 봅니다그렇잖아도 부동산 가격 자꾸 올라가는게 정부정책이 잘못 되었다고 봅니다부동산은 그냥 내버려 두세요  좀건들수록 역효과네요',
 '2']

In [ ]:
## Setting parameters
max_len = 500
batch_size = 1
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

#BERTDataset 클래스 이용, TensorDataset으로 만들어주기
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/drive/My Drive/data/KoBERT-master/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
#배치 및 데이터로더 설정
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


---

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3, ##주의: 클래스 수 바꾸어 주세요!##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device) #gpu

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 학습

In [ ]:
for e in range(num_epochs):
    torch.cuda.empty_cache()
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        # copiedOut = out.data.cpu().numpy()
        # del out
        # torch.cuda.empty_cache()

        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} validation acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/30000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.7354711890220642 train acc 1.0
epoch 1 batch id 201 loss 0.8126379251480103 train acc 0.31343283582089554
epoch 1 batch id 401 loss 1.281842589378357 train acc 0.3266832917705736
epoch 1 batch id 601 loss 1.081777811050415 train acc 0.3227953410981697
epoch 1 batch id 801 loss 1.039162039756775 train acc 0.33208489388264667
epoch 1 batch id 1001 loss 1.1465102434158325 train acc 0.34865134865134867
epoch 1 batch id 1201 loss 1.2852320671081543 train acc 0.37468776019983346
epoch 1 batch id 1401 loss 0.5980306267738342 train acc 0.4125624553890079
epoch 1 batch id 1601 loss 1.1266365051269531 train acc 0.4497189256714553
epoch 1 batch id 1801 loss 0.15847471356391907 train acc 0.48972792892837314
epoch 1 batch id 2001 loss 0.012309649959206581 train acc 0.5242378810594702
epoch 1 batch id 2201 loss 0.010046156123280525 train acc 0.546569741026806
epoch 1 batch id 2401 loss 0.06768465787172318 train acc 0.5685131195335277
epoch 1 batch id 2601 loss 0.01116280630

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 10.5 GB  | Proc size: 5.3 GB
GPU RAM Free: 4184MB | Used: 7257MB | Util  63% | Total 11441MB


---

# 테스트
- category 임의로 생성

In [ ]:
new_test = nlp.data.TSVDataset('/content/drive/MyDrive/data/test.tsv', field_indices=[1,2], num_discard_samples=1)
test_set = BERTDataset(new_test , 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=4)

In [ ]:
pred = []
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)): 
  token_ids = token_ids.long().to(device) 
  segment_ids = segment_ids.long().to(device) 
  valid_length= valid_length 
  out = model(token_ids, valid_length, segment_ids)
  prediction = out.cpu().detach().numpy().argmax()
  print(batch_id + "번째 문장의 분류 예측값은" + prediction + "입니다.")
  pred.append(prediction)